#Errors in serial dilution: How can dispensing technology impact assay data?

####[Sonya M. Hanson and John D. Chodera](http://www.choderalab.org/) Computational Biology Program, Memorial Sloan Kettering Cancer Center, New York NY 10065

'''
Introductory pre-ramble.
'''

In [6]:
import numpy as np

###1. Calculating IC50 for EphB4 system from assay data.

Now lets see how these errors relate to IC50 results. Let's assume EphB4 as our system, with a Km for ATP by EphB4 of 1.71 uM ([EphB4 DataSheet](http://www.proqinase.com/kinase-database/pdfs/2843.pdf)). Note that we're assuming that EphB4 obeys Michaelis-Menten kinetics here, and that V0/Vmax is what is measured.

In [18]:
# Km of ATP for EphB4 (M)
Km = 1.71e-6
# arbitrary compound Ki (M)
true_Ki = 10e-9

# ATP concentration (M)
substrate_concentration = 4e-6
# EphB4 assay concentration (M)
enzyme_concentration = 6e-6

# volume of compound dilution added to assay (L)
compound_volume = 2.0e-6
# volume of enzyme assay mix added to assay (L)
mix_volume = 10.0e-6

In [ ]:
#skeleton place holder
def perfect_dispense(compound_volume, mix_volume, actual_concentrations, perfect_pipetting_model)

In [19]:
def competitive_inhibition(substrate_concentration, inhibitor_concentration, enzyme_concentration, Ki, Km):
    V0_over_Vmax = substrate_concentration / (Km*(1 + inhibitor_concentration/Ki) + substrate_concentration)
    return V0_over_Vmax

In [22]:
activity = np.zeros([ndilutions], np.float64)
for i in range(ndilutions):
    activity[i] = competitive_inhibition(substrate_concentration, assay_compound_concentrations[i], enzyme_concentration, true_Ki, Km)


###2. Adding in inaccuracy and imprecision for Tecan Genesis.

In [ ]:
ndilutions = 8

In [17]:
def dilution_series(V0, C0, Vtransfer, Vbuffer, ndilutions):
    # Initialize concentrations and volumes with zero vectors with C0 and V0 as first value
    concentrations = numpy.zeros([ndilutions], numpy.float64)
    volumes = numpy.zeros([ndilutions], numpy.float64)
    concentrations[0] = C0
    volumes[0] = V0
    
    # and Vbuffer as initial volume for all but first.
    for n in range(1,ndilutions):
        volumes[n] = Vbuffer
        
    # Create dilution series.
    for n in range(1,ndilutions):
        concentrations[n] = concentrations[n-1] * Vtransfer / (Vtransfer + Vbuffer)
        volumes[n] += Vtransfer
        volumes[n-1] -= Vtransfer
        
    # Remove Vtransfer from last.
    volumes[ndilutions-1] -= Vtransfer
    
    return [volumes, concentrations]

In [ ]:
def ROBOT_dilution_series(V0, C0, Vtransfer, Vbuffer, ndilutions, pipetting_model):
    
    # use pipetting error function
    [transfer_inaccuracy, transfer_imprecision] = pipetting_model(Vtransfer)    
            
    # define (b) imprecision of volume transfer operations (including initial volumes)
    transfer_bias = transfer_inaccuracy * normal()
    
    # Initialize concentrations and volumes with zero vectors with C0 and V0 as first value
    actual_concentrations = numpy.zeros([ndilutions], numpy.float64)
    actual_volumes = numpy.zeros([ndilutions], numpy.float64)
    actual_concentrations[0] = C0
    actual_volumes[0] = V0
    
    # and Vbuffer as initial volume for all but first.
    Vbuffer_actual = Vbuffer * ((1+transfer_bias) + transfer_imprecision*normal())
    for n in range(1,ndilutions):
        actual_volumes[n] = Vbuffer_actual
    
    # Create dilution series.
    for n in range(1,ndilutions):
        Vtransfer_actual = Vtransfer * ((1+transfer_bias) + transfer_imprecision*normal())
        actual_concentrations[n] = \
            actual_concentrations[n-1] * Vtransfer_actual / (Vtransfer_actual + Vbuffer_actual)
        actual_volumes[n] += Vtransfer_actual
        actual_volumes[n-1] -= Vtransfer_actual
    Vtransfer_actual = Vtransfer * ((1+transfer_bias) + transfer_imprecision*normal())
    
    # Remove Vtransfer from last.
    actual_volumes[ndilutions-1] -= Vtransfer_actual
    
    return [actual_volumes, actual_concentrations]

In [23]:
#first CV plot

In [ ]:
#first bias blot

In [ ]:
#first non-perfect IC50 plot

###3. Compare this to inaccuracy and imprecision for LabCyte Echo.

In [ ]:
def echo_assay_dispense(C0, mix_volume, backfill_volume, dispense_volumes):
    inaccuracy = 0.10
    imprecision = 0.08

    ndilutions = len(dispense_volumes)
    echo_volume = numpy.zeros([ndilutions], numpy.float64)
    echo_concentration = numpy.zeros([ndilutions], numpy.float64)
    bias = inaccuracy * normal()
    
    for i in range(ndilutions):
        compound_volume_intended = dispense_volumes[i]
        backfill_volume_intended = backfill_volume - compound_volume_intended
        
        compound_volume_dispensed = compound_volume_intended * ((1+bias) + imprecision*normal())
        backfill_volume_dispensed = backfill_volume_intended * ((1+bias) + imprecision*normal())
        
        echo_volume[i] = mix_volume + backfill_volume_dispensed + compound_volume_dispensed
        echo_concentration[i] = C0 * compound_volume_dispensed / echo_volume[i]

    return [echo_volume, echo_concentration]

###4. Add in dilution effect for Tecan Genesis liquid handler.

In [ ]:
def DILUTE_ROBOT_dilution_series(V0, C0, Vtransfer, Vbuffer, ndilutions, pipetting_model):
    
    # use pipetting error function
    [transfer_inaccuracy, transfer_imprecision] = pipetting_model(Vtransfer)    
            
    # define (b) imprecision of volume transfer operations (including initial volumes)
    transfer_bias = transfer_inaccuracy * normal()
    
    # Initialize concentrations and volumes with zero vectors with C0 and V0 as first value
    actual_concentrations = numpy.zeros([ndilutions], numpy.float64)
    actual_volumes = numpy.zeros([ndilutions], numpy.float64)
    actual_concentrations[0] = C0
    actual_volumes[0] = V0
    
    # and Vbuffer as initial volume for all but first.
    Vbuffer_actual = Vbuffer * ((1+transfer_bias) + transfer_imprecision*normal())
    for n in range(1,ndilutions):
        actual_volumes[n] = Vbuffer_actual
    
    # Create dilute dilution series 
    # Observe only change is in actual_concentrations[n].
    for n in range(1,ndilutions):
        Vtransfer_actual = Vtransfer * ((1+transfer_bias) + transfer_imprecision*normal())
        actual_concentrations[n] = \
             actual_concentrations[n-1] * Vtransfer_actual / (Vtransfer_actual + Vbuffer_actual)\
             * (1+dilution_function(Vtransfer))
        actual_volumes[n] += Vtransfer_actual 
        actual_volumes[n-1] -= Vtransfer_actual
    Vtransfer_actual = Vtransfer * ((1+transfer_bias) + transfer_imprecision*normal())
    
    #Remove Vtransfer from last.
    actual_volumes[ndilutions-1] -= Vtransfer_actual
    
    return [actual_volumes, actual_concentrations]

###5. Now compare Tecan Genesis and LabCyte Echo errors.

###6. The HP D300 as an alternative digital dispensing device.